In [1]:
import numpy as np
import pandas as pd
from queue import Queue
import networkx as nx
import plotly.graph_objects as go
import time
from memory_profiler import profile, memory_usage
from matplotlib import pyplot as plt
from PIL import Image

In [2]:
grid_original = np.asarray(pd.read_csv('./labyrinth.csv').iloc[:, 1:])

grid_original.shape

(201, 201)

In [3]:
initial_position = np.asarray(pd.read_csv('./initial_position.csv').iloc[:, 1:])[:, 0]
last_position = np.asarray(pd.read_csv('./last_position.csv').iloc[:, 1:])[:, 0]

initial_position = (initial_position[0], initial_position[1])
last_position = (last_position[0], last_position[1])

initial_position, last_position

((35, 0), (41, 200))

In [4]:
initial_grid = grid_original.copy()
initial_grid[initial_position[0], initial_position[1]] = 'X'

objective_grid = grid_original.copy()
objective_grid[last_position[0], last_position[1]] = 'X'

In [5]:
operations = [
                [ 0, -1], #left
                [-1,  0], #up
                [ 1,  0], #down
                [ 0,  1] #right
             ]

In [6]:
class Node:
    def __init__(self, value, parent=None):
        self.value = value
        self.children = []
        self.parent = parent

    def add_child(self, child):
        node = Node(child, parent=self)
        self.children.append(node)
        return node

class Tree:
    def __init__(self, root):
        self.root = root
         
def find_path_to_root(objective_node):
    path = []
    current_node = objective_node
    while current_node is not None:
        path.insert(0, current_node.value)
        current_node = current_node.parent
    return path

In [7]:
def generate_children(grid, operations, position):
    def change_position(operation):
        if grid[position[0] + operation[0], position[1] + operation[1]] != '#':
            return (position[0] + operation[0], position[1] + operation[1])
        
        else:
            return None
    
    children = []
    grid[position[0], position[1]] = 'X'
    
    '''
        Left
    '''
    if position[1] != 0:
        new_position = change_position(operations[0])
        
        if new_position is not None:
            children.append(new_position)
    
    '''
        Up
    '''
    if position[0] != 0:
        new_position = change_position(operations[1])
        
        if new_position is not None:
            children.append(new_position)
        
    '''
        Down
    '''
    if position[0] != grid.shape[0] - 1:
        new_position = change_position(operations[2])
        
        if new_position is not None:
            children.append(new_position)
        
    '''
        Right
    '''
    if position[1] != grid.shape[1] - 1:
        new_position = change_position(operations[3])
        
        if new_position is not None:
            children.append(new_position)
        
    return children

In [8]:
def construct_solution_BFS_labyrinth(tree):
    nodes_to_expand = Queue()
    nodes_to_expand.put(tree.root)
    visited_nodes = []

    found = False
    path = []

    while not found:
        current_node = nodes_to_expand.get()
        current_position = current_node.value

        visited_nodes.append(current_position)
        children = generate_children(grid, operations, current_position)
        
        for c in children:
            if c not in visited_nodes:
                child = current_node.add_child(c)
                nodes_to_expand.put(child)

                if np.array_equal(c, last_position):
                    found = True
                    path = find_path_to_root(child)
                    visited_nodes.append(last_position)
                    break
                    
    return tree, path, visited_nodes

In [9]:
grid = initial_grid.copy()

In [10]:
start_time = time.time()

root = Tree(Node(initial_position))
tree, solution_path, visited_nodes = construct_solution_BFS_labyrinth(root)

time.time() - start_time

7.208009719848633

In [11]:
#solution_path

In [12]:
def movements_to_solution(solution_path):
    movements = []
    operations = { (0, -1):'left', (-1, 0): 'up', (1, 0): 'down', (0, 1): 'right' }
    
    for i in range(len(solution_path) - 1):
        position_1 = solution_path[i]
        position_2 = solution_path[i + 1]
    
        movements.append(operations[((position_2[0] - position_1[0]),(position_2[1] - position_1[1]))])
        
    return movements

In [13]:
movements = movements_to_solution(solution_path)

In [14]:
#movements